In [ ]:
%run -i ../startup.py
ENTITY_TYPE = 'event'

Wikidata entities:

https://www.wikidata.org/wiki/Q182832 concert

https://www.wikidata.org/wiki/Q1573906 concert tour




Wikidata properties:

https://www.wikidata.org/wiki/Property:P276 location

https://www.wikidata.org/wiki/Property:P580 start time

https://www.wikidata.org/wiki/Property:P585 point in time

https://www.wikidata.org/wiki/Property:P710 participant

https://www.wikidata.org/wiki/Property:P1651 youtube ID

https://www.wikidata.org/wiki/Property:P6423 MusicBrainz event ID





examples

https://www.wikidata.org/wiki/Q898154

https://www.wikidata.org/wiki/Q25408640

## Events from Wikidata

In [ ]:
sparql("""
SELECT (COUNT(?event) AS ?cnt)
WHERE {
  ?event wdt:P31 wd:Q182832 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
""")

In [ ]:
# entity types
sparql("""
SELECT distinct (count(?event) as ?cnt) ?ins ?insLabel
WHERE {
  ?event wdt:P31 ?ins;
    wdt:P6423 ?mbid.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
group by ?ins ?insLabel
order by DESC(?cnt)
""")

In [ ]:
# linked to MB event
links_from_wd = sparql("""
SELECT (?event AS ?wd) ?mbid ?eventLabel
WHERE {
  ?event wdt:P6423 ?mbid .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
ORDER BY ASC(?eventLabel)
""")
links_from_wd.rename(columns={'eventLabel': 'name'}, inplace=True)


print('Count:', len(links_from_wd))
display_df(links_from_wd.head())

Event before 1920:

In [ ]:
sparql("""
SELECT (?event AS ?wd) ?date ?eventLabel ?mbidam

WHERE {
  ?event wdt:P31* wd:Q182832 .
  OPTIONAL { ?event wdt:P6423 ?mbid . }
  ?event wdt:P585 ?date .
  FILTER (year(?date) < 1920) .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
ORDER BY ASC(?date)
""")

## Events from MusicBrainz with wikidata links

In [ ]:
links_from_mb = sql("""
SELECT
    url.url AS wd,
    event.gid AS mbid,
    event.name
FROM event
JOIN l_event_url AS leu ON leu.entity0 = event.id
JOIN url                ON leu.entity1 = url.id
WHERE
    url.url LIKE '%%wikidata.org%%'
ORDER BY event.name
;
""")
links_from_mb.wd = links_from_mb.wd.apply(lambda s: s.split('/')[-1])
links_from_mb.mbid = links_from_mb.mbid.apply(str)

print('Count:', len(links_from_mb))
display_df(links_from_mb.head())

## Duplicate links

In [ ]:
duplicate_wd = links_from_mb[[
    'wd', 'mbid', 'name']].groupby('wd').filter(
    lambda row: len(row.mbid) > 1).sort_values('wd')

print('Count:', len(duplicate_wd))
display_df(duplicate_wd.head())

In [ ]:
duplicate_mb = links_from_mb[['wd', 'mbid', 'name']].groupby('mbid').filter(
    lambda row: len(row.mbid) > 1).sort_values('mbid')

print('Count:', len(duplicate_mb))
display_df(duplicate_mb.head())

## Data alignment

In [ ]:
merge = pd.merge(links_from_wd, links_from_mb, 
                 on=['wd', 'mbid'], suffixes=('_wd', '_mb'),
                 how='outer', indicator=True)
display_df(merge.head())

In [ ]:
# link in mb but missing in wd
links_to_add_to_wd = merge.loc[lambda x : x['_merge']=='right_only'][[
    'name_mb', 'mbid', 'wd']]

print('Count:', len(links_to_add_to_wd))
display_df(links_to_add_to_wd.head())

In [ ]:
# link in wd but missing in mb
links_to_add_to_mb = merge.loc[lambda x : x['_merge']=='left_only'][[
    'name_wd', 'wd', 'mbid']]
links_to_add_to_mb['edit_link'] = links_to_add_to_mb.apply(
    mb_event_edit_wd_link, axis=1)

print('Count:', len(links_to_add_to_mb))
display_df(links_to_add_to_mb.head())

## Report

In [ ]:
import jinja2

template = jinja2.Template("""
<!doctype html>

<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Alignment of MusicBrainz and Wikidata Events</title>
    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
  </head>

  <body style="margin: 20px;">
    <h1>Alignment of MusicBrainz and Wikidata Events</h1>

    <p>Latest MB database update: {{ MB_DATABASE_VERSION }}</p>
    <p>Latest update: {{ date.today().isoformat() }}</p>

    <ol>
      <li>
        <a href="#wddup">MusicBrainz entities sharing a Wikidata link</a>
        ({{ duplicate_wd.shape[0] }} rows)
      </li>
      <li>
        <a href="#mbdup">Wikidata entities sharing a MusicBrainz link</a>
        ({{ duplicate_mb.shape[0] }} rows)
      </li>
      <li>
        <a href="#wd2mb">Add missing Wikidata links to MusicBrainz</a>
        ({{ links_to_add_to_mb.shape[0] }} rows)
      </li>
      <li>
        <a href="#mb2wd">Add missing MusicBrainz links to Wikidata</a>
        ({{ links_to_add_to_wd.shape[0] }} rows)
      </li>
    </ol>
    
    <h2 id="wddup">MusicBrainz entities sharing a Wikidata link</h2>
    {{ df_to_html(duplicate_wd) }}

    <h2 id="mbdup">Wikidata entities sharing a MusicBrainz link</h2>
    {{ df_to_html(duplicate_mb) }}

    <h2 id="wd2mb">Add missing Wikidata links to MusicBrainz</h2>
    {{ df_to_html(links_to_add_to_mb) }}

    <h2 id="mb2wd">Add missing MusicBrainz links to Wikidata</h2>
    {{ df_to_html(links_to_add_to_wd) }}
  </body>
</html>
""")

with open('../docs/wd-events-report.html', 'w') as f:
    f.write(template.render(**globals())
            .replace('&lt;', '<').replace('&gt;', '>')
            .replace('class="dataframe"', 'class="table table-striped table-hover table-sm"')
            .replace('thead', 'thead class="thead-light"'))